## Re-Writing Preprocessing Module

In [161]:
#logger.debug('Inside Logging Function')
#logger.info('info Inside Logging Function')
#logger.warning('warn Inside Logging Function')
#logger.error('error Inside Logging Function')
#logger.critical('critical Inside Logging Function')
#logger.info('--------------- TESTING ENDS ---------------------- ')

### Environmental Setup Scripts

In [210]:
!pip --version

pip 19.3 from /home/alari/anaconda3/envs/ec_final/lib/python3.6/site-packages/pip (python 3.6)


In [1]:
from fastai import *
from fastai.text import *
import os
import pandas as pd
import cloudpickle as cp
from urllib.request import urlopen # or maybe use urllib2
from tqdm import tqdm
import logging
from time import strftime, localtime
import numpy as np
from sklearn.model_selection import train_test_split
import pickle


In [163]:
def define_paths(parentpath):
    PARENT_PATH = parentpath
    if "kwargs" not in locals():
        kwargs = {}
    #LOG_PATH = os.path.relpath("logs") relative_path = os.path.relpath(path, start) 
    LOG_PATH = os.path.join(PARENT_PATH,"logs")
    kwargs = define_logger(log_dir=LOG_PATH,**kwargs)
    raw_data_url = "https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/data_on_public/full/products.csv"
    #################### MODEL GOES ALWAYS TOGETHER WITH SPECIFIC LEARNER AND DATASET
    MODEL_NAME1 = "3_features_80pc.pkl"
    logger.info(f"Model 1 is defined as {MODEL_NAME1}")
    model_1_feature_names = ["combined_name_description", "brand", "provider"]
    kwargs["model1.feature_names"] = model_1_feature_names
    logger.info(f"Model 1 feature names are {model_1_feature_names}")
    LEARNER_REMOTE_PATH1 ="https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/models/3_features_80pc.pkl"
    logger.info(f"Model 1 remote path is defined as {LEARNER_REMOTE_PATH1}")
    MODEL_FOLDER=os.path.join(PARENT_PATH,"models")
    make_sure_path_exists(MODEL_FOLDER)
    logger.info(f"Model folder is defined as {MODEL_FOLDER}")
    LOCAL_MODEL_FULL_PATH = os.path.join(MODEL_FOLDER,MODEL_NAME)
    BASE_DATA_PATH = os.path.join(PARENT_PATH,"data")
    BASE_DATA_FILE = os.path.join(BASE_DATA_PATH,"products.csv")
    make_sure_path_exists(BASE_DATA_PATH)
    kwargs["base_data_path"] = BASE_DATA_PATH
    kwargs["fulldata"] = raw_data_url
    logger.info(f"Raw data remote path is defined as {raw_data_url}")
    kwargs["model1.3_features"]=LEARNER_REMOTE_PATH1
    kwargs["logger"] = logger
    kwargs["base_data_file"] = BASE_DATA_FILE
   # base_data_path = os.path.abspath(os.path.join('data'))
    #kwargs_paths["base_data_path"] = base_data_path
    return kwargs


def define_logger(log_dir,log_to_this_file="e_commerce_logger",log_to_file=False,**kwargs):
    if "kwargs" not in locals():
        kwargs = {}
    make_sure_path_exists(log_dir)
    log_to_this_file = os.path.join(log_dir, log_to_this_file)
    log_to_this_file = log_to_this_file + strftime("%Y_%m_%d_%H_%M", localtime()) + ".log"
    logger = logging.getLogger('e_commerce_project_logger')
    logger.setLevel(logging.DEBUG)
    # create file handler which logs even debug messages
    if log_to_file == True:
        fh = logging.FileHandler(log_to_this_file)
        fh.setLevel(logging.DEBUG)
        # create console handler with a higher log level
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        logger.addHandler(fh)
        print(f"Logging all updates to {log_to_this_file} this file")
        kwargs["logger_path"] = log_to_this_file
    kwargs["logger"] = logger
    return kwargs

def download_from_url(local_path,remote_path,mode="data",**kwargs): 
    logger = kwargs.get("logger")
    logger.info(f"Starting to download {mode} from {remote_path} remote resource")
    if mode == "data":
        FILENAME = "products.csv"
        folder_to_check = kwargs["base_data_path"]
                
    elif mode == "model":
        FILENAME = MODEL_NAME
        folder_to_check = MODEL_FOLDER
        
    if FILENAME not in os.listdir(folder_to_check):
        logger.debug(f"It seems the {mode} hasn't been downloaded yet, so we will do so now: \n")
        logger.debug(f"=================DOWNLOADING THE {mode} ==================")
        filedata = urlopen(remote_path)
        datatowrite = filedata.read()
        with open(local_path, 'wb') as f:
            f.write(datatowrite)

        if FILENAME not in os.listdir(folder_to_check):
            logger.warn(f"It seems {mode} download didn't work, please check the function!")
                
    else:
        logger.info(f"{mode} is present in {folder_to_check}, proceed normally!")



In [164]:
PARENT_PATH = "/home/alari/pCloudDrive/Public Folder/ecommerce/e_commerce_hierarchical/e_commerce_project/code/using_predictions/submission/runtime"
make_sure_path_exists(PARENT_PATH)
os.chdir(PARENT_PATH)  # #os.path.pardir,
print(f"Switched working directory to {PARENT_PATH}, STARTING PROGRAM")

Switched working directory to /home/alari/pCloudDrive/Public Folder/ecommerce/e_commerce_hierarchical/e_commerce_project/code/using_predictions/submission/runtime, STARTING PROGRAM


### Test to See that Data Downloading Works

In [165]:

#download_from_url(mode="data",local_path=kwargs.get("base_data_file"),remote_path = kwargs.get("fulldata"),**kwargs)

In [166]:
# test with this smaller link https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/data/val.csv 

### PREPROCESSING FUNCTIONS DEFINED HERE

In [191]:
def replace_missing_cols_with_empty(df,cols):
    df.loc[:, cols] = df.loc[:, cols].replace(np.nan, '', regex=True)
    return df

def drop_if_exists(df, cols):
	for col in cols:
		if col in df.columns:
			df = df.drop(col, axis=1)
	return df

def text_cleaner(text):
	rules = [
		{r'>\s+': u'>'},  # remove spaces after a tag opens or closes
		{r'\s+': u' '},  # replace consecutive spaces
		{r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
		{r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
		{r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
		{r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
		{r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
		{r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
		{r'^\s+': u''} # remove spaces at the beginning
    ]
	for rule in rules:
		for (k, v) in rule.items():
			regex = re.compile(k)
			text = regex.sub(v, text)
	text = text.rstrip()
	return text.lower()

def savePickle(data, picklefile):
	with open(picklefile, "wb") as f:
		pickle.dump(data, f)
	f.close()
    
def step_1_data_processing(df,**kwargs):
    logger = kwargs.get("logger")
    
    
    initial_feature_names =  kwargs["initial_feature_names"]
    logger.info(f"Feature names of un-processed raw data are {initial_feature_names} \n")
  

    df["cleaned_name"] = df.progress_apply(lambda row: text_cleaner(str(row["name"]).lower()), axis=1)
    df["cleaned_description"] = df.progress_apply(lambda row: text_cleaner(str(row["description"]).lower()), axis=1)
    df["combined_name_description"] = df.progress_apply(
        lambda row: str(row["cleaned_name"]).lower() + str(row["cleaned_description"]).lower(), axis=1)
    
    model1_feature_names =  kwargs["model1.feature_names"]
    logger.info(f"Retaining only {model1_feature_names} out of all features. Also replacing NAN with empty.")

    df = replace_missing_cols_with_empty(df,cols=model1_feature_names)
    target_with_feature_names = [i for i in itertools.chain(model1_feature_names, ["mapped_id"])]
    df = df.loc[:,target_with_feature_names]
    logger.info("Category preprocessing function finished!")
    return df

def step_2_save_splits_from_dataframe(df,feature_names=["combined_name_description", "brand", "provider", "summary"], \
                                             target_name=["mapped_id"],**kwargs):
    save_splits_again = kwargs.get("save_splits_again")
    logger = kwargs.get("logger")
    df.loc[:, target_name] = df.loc[:, target_name].astype(int)
    df = df.groupby(target_name).filter(lambda x: len(x) > 1)
    logger.info(f"Target classes with only 1 count have been filtered out. Resulting dataframe has {df.shape[0]} elements")
    features = df.loc[:, feature_names].reset_index(drop=True)
    target = df.loc[:,target_name].reset_index(drop=True)
    time_now = strftime("%Y_%m_%d_%H_%M", localtime())
    logger.info(f"Time now is {time_now}")
    X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.20,
                                                      random_state=42, stratify=target)

    ml_splits = {'X_train': X_train, 'X_val': X_val, 'y_train': y_train, 'y_val': y_val}

    # Here we are going to create the output dictionary to save the machine learning splits in one Pickle file
    train_val_dict = {}
    train_val_dict["train"] = pd.concat([ml_splits.get("X_train"),ml_splits.get("y_train")],axis=1)
    train_val_dict["val"] = pd.concat([ml_splits.get("X_val"),ml_splits.get("y_val")],axis=1)
    logger.info("Header of the train-validation machine learning splits dictionary look like this: \n")
    logger.info(str(apply_fun(train_val_dict,fun=pd.DataFrame.head)))
    #ml_splits_wout_summary_nastat = ut.apply_fun(dict_=ml_splits_wout_summary,fun=pd.DataFrame.isna)
    #na_statistics = ut.apply_fun(dict_=ml_splits_wout_summary_nastat,fun=pd.DataFrame.sum)
    #logger.info(f"The NA Statistics look the following {str(na_statistics)}")
    if save_splits_again == True:
        time_now = strftime("%Y_%m_%d_%H_%M", localtime())
        splits_end = f"ml_splits_{time_now}.pkl"
        splits_path = os.path.join(kwargs_paths.get("base_data_path"), splits_end)
        savePickle(data=training_dict_save, picklefile=splits_path)
        logger.info(f"Saving ML Splits to the path {splits_path}")
    return train_val_dict
                

### PREDICTION FUNCTIONS DEFINED HERE

In [193]:
def apply_fun(dict_: dict, fun, **kwargs):
	transformed_dict = dict((k, fun(v, **kwargs)) for k, v in dict_.items())
	return transformed_dict

def select_features_from_data(data,feature_names=["recombined_name_description", "brand", "provider"]):
    features = data.loc[:, feature_names].reset_index(drop=True)
    return features


def get_one_prediction(feature_row):
    one_prediction_unformatted = model.predict(feature_row)[0]
    print(one_prediction_unformatted)
    one_prediction_int = int(one_prediction_unformatted)
    return one_prediction_int



def obtain_predictions_in_loop(features):
        for i,feature_row in tqdm(features.iterrows(),total=features.shape[0]):
        #for i,row in (data_full.iterrows()):
            prediction= get_one_prediction(feature_row)
            print(f"Feature row {i} looks like this: \n")
            print(f"{str(feature_row.values)}\n")
            print(f"The corresponding prediction row {i} is: \n")
            print(prediction)
            if i==10:
                break

In [169]:

#import logging
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
#logging.debug("test")

## Running the Main Program Here

In [194]:
kwargs = define_paths(parentpath=PARENT_PATH)
logger=kwargs.get("logger")
kwargs["parentpath"] = PARENT_PATH # parentpath is the project path from which the code is run
kwargs["skip"] = ""#"0 2 " --- data pipeline is organized as sequential steps. Write here the steps you wish to skip
kwargs["amountlines"]=int(-1) # -1 corresponds to all lines of raw data read in from CSV, otherwise the integer specifies how many lines are are read in from beginning
kwargs["save_splits_again"] = False # whether to re-save the machine learning train-test split file again


def main(**kwargs):
    """This is the caller function of the data pipeline, producing predictions of E-Commerce Categories from a defined set of features."""
    step1=step2={}
    # data_object is the dictionary containing outputs of data pipeline steps
    data_object = {"step1":step1,"step2":step2} 
    logger = kwargs.get("logger")
    data_object["kwargs_main"] = kwargs
    #print(main.__doc__)
    #parser = argparse.ArgumentParser()
    #parser.add_argument("-skip","--skip-steps", default="2", type=str,help="Steps to skip separated by space")
    #args = parser.parse_args()
    skip_steps = kwargs.get("skip").split(' ') #     df = kwargs.get("raw_data")

    logger.info(" ----------- STEP 1 : DATA LOADING AND PROCESSING  --------------")

    if "data_read" not in skip_steps:
        # model1.3_features
        download_from_url(mode="data",local_path=kwargs.get("base_data_file"),remote_path = kwargs.get("fulldata"),**kwargs)
        raw_data_local_path = kwargs.get("base_data_file")
        print(f"THE RAW DATA LOCAL PATH FROM KWARGS IS {raw_data_local_path}")
        amountlines = kwargs.get("amountlines")
        if amountlines != -1:
            rows_to_keep = list(range(amountlines))
            df = pd.read_csv(raw_data_local_path, skiprows=lambda x: x not in rows_to_keep)
            logger.info(f"----- READING {rows_to_keep} lines of  DATA FROM  path {raw_data_path} COMPLETED ------------------ ")
        else:
            df = pd.read_csv(raw_data_local_path)
            logger.info(f"----- READING FULL DATA FROM  path {raw_data_local_path} COMPLETED ------------------ ")
    
    cols_to_drop = ['ean', 'url', 'price', './._prods.csv']
    logger.info(f"Going to drop these columns if they exist: {cols_to_drop}\n")
    df = drop_if_exists(df, cols=cols_to_drop)
    initial_feature_names = [x for x in df.columns.tolist() if x!="mapped_id"]
    kwargs["initial_feature_names"] = initial_feature_names
    
    if df is None:
        print("THERE IS NO DATA TO BE READ, PROGRAM STOPPING!")
        return

    tqdm.pandas(desc='PROGRESS>>>')
    data_1 = step_1_data_processing(df,**kwargs)

    step1["data"] = data_1
    data_object["data_read"] = step1
    logger.info(str(data_1.head()))
    logger.info(" ----------- STEP 1 COMPLETE : DATA LOADING AND PROCESSING  --------------")

    logger.info(" ----------- STEP 2 BEGINS : DATA PRE-PROCESSING  --------------")

    """ Preferred mode : read data from disk"""
    if "data_process" not in skip_steps:
        data_2 = step_2_save_splits_from_dataframe(df=data_1,feature_names=["combined_name_description", "brand", "provider"],**kwargs)
    else:
        data_2 = []
        #data_2 = read_data_splits_pickle_from_cloud(pickle_url="https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/data/final_splits.pkl")
        #DATA_to_validate = DATA_splits["val"]
    step2["data"] = data_2
    data_object["data_process"] = step2


    logger.info(f" ----------- STEP 2, preprocessing, ENDED.  --------------")

    logger.info(" ----------- STEP 3 : DOWNLOADING AND LOADING PRETRAINED HIGH DIMENSIONAL CLASSIFICATION MODEL  --------------")
    if "model" not in skip_steps:
        model =  download_and_load_model()
    data_object["model"] = model 

    if "predictions" not in skip_steps:
        DATA_to_validate = data_2["val"]
        FEATURES_to_validate = select_features_from_data(data=DATA_to_validate,feature_names=["combined_name_description", "brand", "provider"])
        # FEATURES_to_validate.reset_index(drop=True)
        obtain_predictions_in_loop(features=FEATURES_to_validate)
        data_object["features"] = FEATURES_to_validate

    #data_object["predictions"] = predictions # save the predictions here somewhere
    return data_object


data_pipeline_object = main(**kwargs)















  0%|          | 0/111363 [00:00<?, ?it/s]

Model 3_features_80pc.pkl loaded!











  0%|          | 1/111363 [00:00<12:03:01,  2.57it/s]

212
Feature row 0 looks like this: 

combined_name_description    neulepusero roxy crazy cityneulepusero roxy cr...
brand                                                                     Roxy
provider                                                               Spartoo
Name: 0, dtype: object

The corresponding prediction row 0 is: 

71











  0%|          | 2/111363 [00:00<11:20:37,  2.73it/s]

...

187
Feature row 7 looks like this: 

combined_name_description    kengät asics gel-lyte iiikengät asics gel-lyte...
brand                                                                    Asics
provider                                                               Spartoo
Name: 7, dtype: object

The corresponding prediction row 7 is: 

55











  0%|          | 9/111363 [00:04<15:26:41,  2.00it/s]

187
Feature row 8 looks like this: 

combined_name_description    sandaalit el naturalista leavessandaalit el na...
brand                                                           El Naturalista
provider                                                               Spartoo
Name: 8, dtype: object

The corresponding prediction row 8 is: 

55











  0%|          | 10/111363 [00:04<14:02:22,  2.20it/s]

1604
Feature row 9 looks like this: 

combined_name_description    get farkkutakki denimtakki musta tiger of swed...
brand                                                    Tiger of Sweden Jeans
provider                                                                 Boozt
Name: 9, dtype: object

The corresponding prediction row 9 is: 

440
203
Feature row 10 looks like this: 

combined_name_description    hooded water-repellent coat toppatakki harmaa ...
brand                                                               Mango Kids
provider                                                                 Boozt
Name: 10, dtype: object

The corresponding prediction row 10 is: 

65


In [196]:
model = data_pipeline_object.get("model")
model

RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: /home/alari/pCloudDrive/Public Folder/ecommerce/e_commerce_hierarchical/e_commerce_project/code/using_predictions/submission/models;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /home/alari/pCloudDrive/Public Folder/ecommerce/e_commerce_hierarchical/e_commerce_project/code/using_predictions/submission/models;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1150, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1150, 1150, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1150, 400, batch_first=True)
        )
      )
      (input_dp): RNNDro

# Using Predictions : Step-by-Step

## Preprocessing the Data for Loading it Into the Model

In [134]:
MODEL_FOLDER=os.path.abspath("models")

def make_sure_path_exists(dir):
    if not os.path.exists(dir):
        try:
            os.makedirs(dir)
            print(f"{dir} path didn't exist, created! Proceed now!")
        except OSError as exception:
                print("Directory Creation Exception")


        
make_sure_path_exists(MODEL_FOLDER)
LEARNER_REMOTE_PATH ="https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/models/3_features_80pc.pkl"
MODEL_NAME = "3_features_80pc.pkl"
LOCAL_MODEL_FULL_PATH = os.path.join(MODEL_FOLDER,MODEL_NAME)

### Getting the Data to Validate

In [49]:
############ Define DATA and FEATURES in single CELL ##################
def read_data_splits_pickle_from_cloud(pickle_url):
    data = cp.load(urlopen(pickle_url))
    return data

DATA_splits = read_data_splits_pickle_from_cloud(pickle_url="https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/data/final_splits.pkl")
DATA_to_validate = DATA_splits["val"]


## Here are the Cells To be Executed One by One

In [12]:
MODEL_FOLDER=os.path.abspath("models")

        
make_sure_path_exists(MODEL_FOLDER)
LEARNER_REMOTE_PATH ="https://filedn.com/lK1VhM9GbBxVlERr9KFjD4B/ecommerce/ulmfit_final_files/models/3_features_80pc.pkl"
MODEL_NAME = "3_features_80pc.pkl"
LOCAL_MODEL_FULL_PATH = os.path.join(MODEL_FOLDER,MODEL_NAME)

/home/alari/pCloudDrive/Public Folder/ecommerce/e_commerce_hierarchical/e_commerce_project/code/using_predictions/submission/models path exists, proceed!


In [42]:
# In this cell, we are going to download and load the model for making predictions later


 
def load_model_from_local_path(model_folder=MODEL_FOLDER,model_name=MODEL_NAME):
    model = load_learner(path=model_folder,
                     file=model_name,**{"num_workers":0}) # OR NUM WORKERS
    if "model" in locals():
        print(f"Model {model_name} loaded!")
    else :
        print("Something went wrong loading the model")
    return model

def download_and_load_model():
    download_from_url(mode="model",local_path=LOCAL_MODEL_FULL_PATH,remote_path=LEARNER_REMOTE_PATH)
    model = load_model_from_local_path()
    return model

model = download_and_load_model()

Model 3_features_80pc.pkl loaded!


In [66]:
def select_features_from_data(data,feature_names=["combined_name_description", "brand", "provider"]):
    features = data.loc[:, feature_names].reset_index(drop=True)
    return features


def get_one_prediction(feature_row):
    one_prediction_unformatted = model.predict(feature_row)[0]
    print(one_prediction_unformatted)
    one_prediction_int = int(one_prediction_unformatted)
    return one_prediction_int



def obtain_predictions_in_loop(features):
        for i,feature_row in tqdm(features.iterrows(),total=features.shape[0]):
        #for i,row in (data_full.iterrows()):
            prediction= get_one_prediction(feature_row)
            print(f"Feature row {i} looks like this: \n")
            print(f"{str(feature_row.values)}\n")
            print(f"The corresponding prediction row {i} is: \n")
            print(prediction)
            if i==10:
                break
            
FEATURES = select_features_from_data(data=DATA_to_validate)
obtain_predictions_in_loop(features=FEATURES)









  0%|          | 0/111363 [00:00<?, ?it/s]



  0%|          | 1/111363 [00:00<11:00:47,  2.81it/s]

212
Feature row 0 looks like this: 

combined_name_description    neulepusero roxy crazy cityneulepusero roxy cr...
brand                                                                     Roxy
provider                                                               Spartoo
Name: 0, dtype: object

The corresponding prediction row 0 is: 

71






  0%|          | 2/111363 [00:00<10:55:35,  2.83it/s]

3553
Feature row 1 looks like this: 

combined_name_description    swedish grace lautanen 21 cm midnatt
brand                                                   Rörstrand
provider                                              KitchenTime
Name: 1, dtype: object

The corresponding prediction row 1 is: 

737






  0%|          | 3/111363 [00:01<13:55:57,  2.22it/s]

...

187
Feature row 7 looks like this: 

combined_name_description    kengät asics gel-lyte iiikengät asics gel-lyte...
brand                                                                    Asics
provider                                                               Spartoo
Name: 7, dtype: object

The corresponding prediction row 7 is: 

55






  0%|          | 9/111363 [00:04<14:11:22,  2.18it/s]

187
Feature row 8 looks like this: 

combined_name_description    sandaalit el naturalista leavessandaalit el na...
brand                                                           El Naturalista
provider                                                               Spartoo
Name: 8, dtype: object

The corresponding prediction row 8 is: 

55






  0%|          | 10/111363 [00:04<12:51:32,  2.41it/s]

1604
Feature row 9 looks like this: 

combined_name_description    get farkkutakki denimtakki musta tiger of swed...
brand                                                    Tiger of Sweden Jeans
provider                                                                 Boozt
Name: 9, dtype: object

The corresponding prediction row 9 is: 

440
203
Feature row 10 looks like this: 

combined_name_description    hooded water-repellent coat toppatakki harmaa ...
brand                                                               Mango Kids
provider                                                                 Boozt
Name: 10, dtype: object

The corresponding prediction row 10 is: 

65
